In [1]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
#from wordcloud import WordCloud
import nltk

from nltk.stem.porter import *
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split as tts

from sklearn.model_selection import StratifiedKFold as SKF

import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

numpy.random.seed(7)

import tweepy as tp
import twitter_credentials as tc

In [2]:
train1 = pd.read_csv('train_E6oV3lV.csv').drop(['id'], axis = 1)

In [3]:
combi = train1
combi.shape

(31962, 2)

In [4]:
combi.drop_duplicates(subset = ['tweet'])
combi.shape

(31962, 2)

In [5]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt  

In [6]:
combi['clean'] = np.vectorize(remove_pattern)(combi['tweet'], "@[\w]*") #removing users
combi['clean'] = combi['clean'].str.replace("[^a-zA-Z#]", " ") #obtaining only words and hashtags
combi['clean'] = combi['clean'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3])) #removing shortwords

In [7]:
tokenized_tweet = combi['clean'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [when, father, dysfunctional, selfish, drags, ...
1    [thanks, #lyft, credit, cause, they, offer, wh...
2                              [bihday, your, majesty]
3                     [#model, love, take, with, time]
4                   [factsguide, society, #motivation]
Name: clean, dtype: object

In [8]:
stopw = set(stopwords.words('english'))

for i in range(tokenized_tweet.shape[0]):
    tokenized_tweet[i] = [w for w in tokenized_tweet[i] if w not in stopw]

stemmer = PorterStemmer()

tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0    [father, dysfunct, selfish, drag, kid, dysfunc...
1    [thank, #lyft, credit, caus, offer, wheelchair...
2                                    [bihday, majesti]
3                           [#model, love, take, time]
4                         [factsguid, societi, #motiv]
Name: clean, dtype: object

In [9]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])

combi['clean'] = tokenized_tweet

In [10]:
tfidf_vectorizer = TfidfVectorizer()
tfidf = tfidf_vectorizer.fit_transform(combi['clean']).toarray()

X = tfidf
Y = combi['label']

#train_tfidf = tfidf#[:trainSize,:]
#test_tfidf = tfidf[trainSize:,:]

#xtrain, xtest, ytrain, ytest = tts(train_tfidf, combi['label'], random_state = 42, test_size = 0.4)

In [11]:
ngram_vect = CountVectorizer(ngram_range = (2,2))
ngram = ngram_vect.fit_transform(combi['clean']).toarray()

In [12]:
max_review_length = 500
#xtrain = sequence.pad_sequences(xtrain, maxlen=max_review_length)
#xtest = sequence.pad_sequences(xtest, maxlen=max_review_length)

In [13]:
top_words = 5000
embeddingVectorLength = 32

#This is for TFIDF

xtrain, xtest, ytrain, ytest = tts(X, Y, random_state = 42, test_size = 0.3)
xtrain = sequence.pad_sequences(xtrain, maxlen = max_review_length)
xtest = sequence.pad_sequences(xtest, maxlen = max_review_length)

model = Sequential()
model.add(Embedding(top_words, embeddingVectorLength, input_length = max_review_length))
model.add(LSTM(200))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
model.fit(xtrain, ytrain, validation_data = (xtest, ytest), epochs = 5, batch_size = 64)

print(model.evaluate(xtest, ytest,  verbose = 0)[1]*100)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 500, 32)           160000    
_________________________________________________________________
lstm (LSTM)                  (None, 200)               186400    
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 346,601
Trainable params: 346,601
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/5
350/350 [==============================] - 46s 106ms/step - loss: 0.2962 - accuracy: 0.9119 - val_loss: 0.2602 - val_accuracy: 0.9287
Epoch 2/5
350/350 [==============================] - 34s 98ms/step - loss: 0.2530 - accuracy: 0.9308 - val_loss: 0.2576 - val_accuracy: 0.9287
Epoch 3/5
350/350 [==============================] - 34s 98ms/step - loss: 0.2558 - accu

In [14]:
X = ngram
Y = combi['label']

xtrain, xtest, ytrain, ytest = tts(X, Y, random_state = 42, test_size = 0.3)
xtrain = sequence.pad_sequences(xtrain, maxlen = max_review_length)
xtest = sequence.pad_sequences(xtest, maxlen = max_review_length)

model = Sequential()
model.add(Embedding(top_words, embeddingVectorLength, input_length = max_review_length))
model.add(LSTM(200))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
print(model.summary())
model.fit(xtrain, ytrain, validation_data = (xtest, ytest), epochs = 5, batch_size = 64)

print(model.evaluate(xtest, ytest,  verbose = 0)[1]*100)

MemoryError: Unable to allocate 21.9 GiB for an array with shape (22373, 131418) and data type int64

In [ ]:
auth = tp.OAuthHandler(tc.consumerKey, tc.consumerSecret)
auth.set_access_token(tc.accessToken, tc.accessTokenSecret)
api = tp.API(auth)

In [ ]:
tweets = api.home_timeline()

for tweet in tweets:
    print(tweet.text)
    print()

In [ ]:
text = tweets[0].text

In [ ]:
text = re.sub(r'http\S+', '', text)